# Parsing the log files into multiple valid JSON files

|Author|Christopher Mortimer|
|---|---|
|Date|2020-06-14|

## Import the json paackage for easy parsing

In [3]:
import json, os

## Format of a txd
TXD is STR's table definition markup language
 
 ```sql
 ROW
     DIM
         RCD "Professions" FACTTABLE "All Professions All Years"
     END DIM
     AXIS_MAP
     END AXIS_MAP
 END ROW
 COLUMN
     DIM
         RCD "Year" FACTTABLE "All Professions All Years"
     END DIM
     AXIS_MAP
     END AXIS_MAP
 END COLUMN
 END TABLE
  ```
  

In [4]:
# JSON check function
def isJson(myjson):
  try:
    json_object = json.loads(myjson)
  except ValueError as e:
    return False
  return True

## the parseFile() function

Parses each log line of a log file and extracts the rows and columns in the TXD cross tabulation.

In [44]:
def parseFile(filename, debugFlag):
  # the JSON file
  f = open("./data/" + env + "/" + filename, "r+")
  ## Loop through each line in the file
  for idx, x in enumerate(f):
    if debugFlag:
      print(idx)
    if isJson(x):
      jsonLine = json.loads(x)
      ## Check the keys in the line, if the line has a txd then parse
      jsonKeys=jsonLine.keys()
      if "txd" in jsonKeys:
        # Get the txd field
        txd=jsonLine["txd"]
        # txd is STR table definition markup, the table part is after \n\nTABLE\n
        if "\n\nTABLE\n" in txd:
          column = ''
          row = ''
          table=txd.split('\n\nTABLE\n')[1]
          # Lets just get the DIMs
          if "ROW\n\tDIM\n\t\tRCD" in table:
            # how many rows?
            #print("rows: " + str(len(table.split('ROW\n\tDIM\n\t\tRCD'))-1))
            row=table.split('ROW\n\tDIM\n\t\tRCD')[1].split("END")[0]
            if "ORDER" in row:
              #print("found order")
              row=row.split("ORDER")[0]
          if "COLUMN\n\tDIM\n\t\tRCD" in table:
            #print("columns: " + str(len(table.split('COLUMN\n\tDIM\n\t\tRCD'))-1))
            column=table.split('COLUMN\n\tDIM\n\t\tRCD')[1].split("END")[0]
            if "Summation Options" in column:
              #print("found summation")
              column=column.split("Summation Options")[0]
            if "ORDER" in column:
              #print("found order")
              column=column.split("ORDER")[0]
          ## Add another two json fields for row and column
          jsonLine['txd'] = 'truncate-save-space'
          jsonLine['row'] = row.strip()
          jsonLine['column'] = column.strip()
      if debugFlag:
        print(jsonLine)
      else:
        ## Create new file
        fProc = open("./data/" + env + "/pythonProc/" + filename + '.' + str(idx) + '.json', "w")
        fProc.write(json.dumps(jsonLine))
        fCsv = open("./data/" + env + '.csv', "a+")
        csvLine = ','.join([str(idx),env,filename,"./data/" + env + "/pythonProc/" + filename + '.' + str(idx) + '.json'+"\n"])
        fCsv.write(csvLine)
    else:
      print('not json') 

In [45]:
## Unit test for parsing a file
env = "test"
filename1 = "2020-05-14-superweb2.log"
parseFile(filename1,0)

## the loopFiles() function

Gets a list of all the files in a directory and calls the parseFile() function on them.

In [46]:
def loopFiles(dirname,debugFlag):
  fCsv = open("./data/" + env + '.csv', "w+")
  fCsv.write('')
  directory = os.fsencode(dirname)
  for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".log"): 
      if debugFlag:
        print(filename)
        parseFile(filename,1)
      else:
        parseFile(filename,0)
    else:
      print(filename + " is not a log file")

In [48]:
## unit test for loopFiles
env = "test"
dirname = "./data/" + env + "/"
loopFiles(dirname,0)

proc is not a log file
pythonProc is not a log file
